In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)  # Set to None to display all rows
import urllib
import psycopg2
from psycopg2.extras import execute_values
from datetime import datetime, timedelta, timezone

from utils import get_request
from db_utils.base_utils import get_db_connection
from db_utils.prop_odds_db_utils import *

In [2]:
# Today's date (EST)
(datetime.now() - timedelta(hours=6)).strftime('%Y-%m-%d')

'2024-12-13'

In [3]:
# game_id = "c1fa7d3f30fdb408b78917509d1633c3"
# data = fetch_game_markets(game_id, 'player_shots_over_under' )
# data
# for sportsbook in data['sportsbooks']:
#     print(sportsbook)
# # Create empty dict to store outcomes by bookie
# outcomes_by_bookie = {}

# for sportsbook in data['sportsbooks']:
#     bookie_key = sportsbook['bookie_key']
#     outcomes_by_bookie[bookie_key] = []
    
#     # Get all outcomes for this sportsbook
#     for outcome in sportsbook['market']['outcomes']:
#         outcomes_by_bookie[bookie_key].append({
#             'timestamp': outcome['timestamp'],
#             'handicap': outcome['handicap'], 
#             'odds': outcome['odds'],
#             'participant_name': outcome['participant_name'],
#             'name': outcome['name'],
#             'description': outcome['description']
#         })

In [4]:
# fetch_and_store_nhl_games('2024-12-11')

In [5]:
# get_nhl_games_from_db('2024-12-11', enable_logging=True)

In [6]:
df = pd.read_csv('data/player_shots_ou.csv')
df['game_id'].unique()

array(['c1fa7d3f30fdb408b78917509d1633c3',
       'ca040c2d8fbe567334ed450ab0a02173',
       'd5a68b58d97f6174dc4c540287ff7e3d',
       'af1551ae76d93d6f65c47cb6b26cfd83',
       'c91477501d6ee060317f9fe60db4117f',
       'ba8a3041639b65d6d085fb72d73b7b98',
       '8fc6f715c673c865fed0d1139b31e261',
       '7fcf37fe5624af32decff26b0cf6e1b1',
       'cebb8cb7b5ffee324b463b8c647ce976',
       'ea5343d38a02fc19a5f435c651d245c9',
       '24083d2c612b7f3c8909dcc6bfd9a4e1',
       '480fa6c594877fff4ce30641dc424fe7',
       '77f4c3b4372e22363655c9862424f6b0',
       'c906c0d63116e928c7813733c806c69a',
       'ba3bf40aeee9beb78786af904b0b7e52'], dtype=object)

In [7]:
# process_game_markets('2024-12-12', 'TOR', 'player_shots_over_under')

In [8]:
# process_nhl_games_for_date('2024-12-10')

In [9]:
# process_nhl_games_for_date('2024-12-07', enable_logging=True)

In [10]:
odds = get_player_shots_ou_odds(player_name='Artturi Lehkonen', query_date='2024-12-10', sportsbook='pinnacle', line=True)
odds

[{'game_id': 'ba8a3041639b65d6d085fb72d73b7b98',
  'sportsbook': 'pinnacle',
  'player': 'Artturi Lehkonen',
  'ou': 'Over',
  'handicap': 2.5,
  'odds': -105,
  'timestamp': datetime.datetime(2024, 12, 11, 0, 7, 10, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=64800)))},
 {'game_id': 'ba8a3041639b65d6d085fb72d73b7b98',
  'sportsbook': 'pinnacle',
  'player': 'Artturi Lehkonen',
  'ou': 'Under',
  'handicap': 2.5,
  'odds': -127,
  'timestamp': datetime.datetime(2024, 12, 11, 0, 7, 10, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=64800)))}]

In [11]:
for odd in odds:
    print(odd['handicap'], odd['odds'])

2.5 -105
2.5 -127


In [13]:
process_game_markets('2024-12-08', 'CGY', 'player_shots_over_under')

In [14]:
get_mismatched_game_ids_with_details(enable_logging=True)

INFO:root:Game IDs only in game_info: [('9d26751d34ceb6f1924a6f816dcce542', 'Pittsburgh Penguins', 'Montréal Canadiens', datetime.datetime(2024, 12, 12, 18, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=64800)))), ('c929ad57dc1f0dfa1ef992b05f392947', 'Los Angeles Kings', 'New Jersey Devils', datetime.datetime(2024, 12, 12, 18, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=64800)))), ('a560c6c2dbf99f539afddfe1b783612d', 'Detroit Red Wings', 'Philadelphia Flyers', datetime.datetime(2024, 12, 12, 18, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=64800)))), ('a0c798d566d9e21c6bbfd0d3c33ce550', 'Washington Capitals', 'Columbus Blue Jackets', datetime.datetime(2024, 12, 12, 18, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=64800)))), ('e0ec5c610f9f5278c24f1596d78464c8', 'Chicago Blackhawks', 'New York Islanders', datetime.datetime(2024, 12, 12, 18, 30, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=64800)))

{'only_in_game_info': [{'game_id': '9d26751d34ceb6f1924a6f816dcce542',
   'away_team': 'Pittsburgh Penguins',
   'home_team': 'Montréal Canadiens',
   'start_timestamp': '2024-12-12'},
  {'game_id': 'c929ad57dc1f0dfa1ef992b05f392947',
   'away_team': 'Los Angeles Kings',
   'home_team': 'New Jersey Devils',
   'start_timestamp': '2024-12-12'},
  {'game_id': 'a560c6c2dbf99f539afddfe1b783612d',
   'away_team': 'Detroit Red Wings',
   'home_team': 'Philadelphia Flyers',
   'start_timestamp': '2024-12-12'},
  {'game_id': 'a0c798d566d9e21c6bbfd0d3c33ce550',
   'away_team': 'Washington Capitals',
   'home_team': 'Columbus Blue Jackets',
   'start_timestamp': '2024-12-12'},
  {'game_id': 'e0ec5c610f9f5278c24f1596d78464c8',
   'away_team': 'Chicago Blackhawks',
   'home_team': 'New York Islanders',
   'start_timestamp': '2024-12-12'},
  {'game_id': 'de19c864744c453ab0185c8d57b2688d',
   'away_team': 'San Jose Sharks',
   'home_team': 'St. Louis Blues',
   'start_timestamp': '2024-12-12'},
  {'